<a href="https://colab.research.google.com/github/SJhawar1010/Reinforcement-Learning/blob/main/Satyam_554_RLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install cmake
!pip install atari-py
!pip install gym
!pip install gym[atari]
!pip install keras-rl2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.6/540.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atari-py: filename=atari_py-0.2.9-cp310-cp310-linux_x86_64.whl size=2872085 sha256=e4c712378345c67b26c34fa562d7741a925cf79199a85a476f351d1d32567f96
  Stored in directory: /root/.cache/pip/wheels/75/6f/04/1f3bf5255580101e16ff487564354dddcdd23ec3b43b775b7a
Successfully built atari-py
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 1.8 MB/s eta 0:00:00


In [2]:
import zipfile
import os

In [3]:
ZFP = "/content/Atari-2600-VCS-ROM-Collection.zip"
EP = "/content/Atari-2600-VCS-ROM-Collection"

In [4]:
if os.path.exists(ZFP):
    with zipfile.ZipFile(ZFP, 'r') as zip_ref:
        zip_ref.extractall(EP)
    print(f"Files extracted to: {EP}")
else:
    print(f"Zip file not found at: {ZFP}")

Files extracted to: /content/Atari-2600-VCS-ROM-Collection


In [5]:
!python -m atari_py.import_roms /content/Atari-2600-VCS-ROM-Collection

copying lost_luggage.bin from /content/Atari-2600-VCS-ROM-Collection/ROMS/Lost Luggage (Airport Mayhem) (1982) (Apollo - Games by Apollo, Larry Minor, Ernie Runyon, Ed Salvo) (AP-2004) [no opening scene] ~.bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/lost_luggage.bin
copying keystone_kapers.bin from /content/Atari-2600-VCS-ROM-Collection/ROMS/Keystone Kapers - Raueber und Gendarm (1983) (Activision, Garry Kitchen - Ariola) (EAX-025, EAX-025-04I - 711 025-725) (PAL).bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/keystone_kapers.bin
copying battle_zone.bin from /content/Atari-2600-VCS-ROM-Collection/ROMS/Battlezone (1983) (Atari - GCC, Michael Feinstein, Patricia Goodson, Brad Rice) (CX2681) ~.bin to /usr/local/lib/python3.10/dist-packages/atari_py/atari_roms/battle_zone.bin
copying galaxian.bin from /content/Atari-2600-VCS-ROM-Collection/ROMS/Galaxian (1983) (Atari - GCC, Mark S. Ackerman, Tom Calderwood, Patricia Goodson, Glenn Parker) (CX2684)

In [6]:
import gym
import random
import atari_py
import numpy as np
import random
import keras
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.layers import *
from collections import deque
print(atari_py.list_games())

/usr/local/lib/python3.10/dist-packages/ale_py/roms/__init__.py:89: DeprecationWarning: Automatic importing of atari-py roms won't be supported in future releases of ale-py. Please migrate over to using `ale-import-roms` OR an ALE-supported ROM package. To make this warning disappear you can run `ale-import-roms --import-from-pkg atari_py.atari_roms`.For more information see: https://github.com/mgbellemare/Arcade-Learning-Environment#rom-management
  ROMS = resolve_roms()
/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:51: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


['tetris', 'galaxian', 'seaquest', 'frogger', 'carnival', 'asterix', 'frostbite', 'solaris', 'journey_escape', 'gopher', 'asteroids', 'ms_pacman', 'laser_gates', 'pacman', 'space_invaders', 'battle_zone', 'yars_revenge', 'boxing', 'road_runner', 'wizard_of_wor', 'time_pilot', 'jamesbond', 'freeway', 'private_eye', 'berzerk', 'pooyan', 'sir_lancelot', 'tutankham', 'pitfall', 'assault', 'mr_do', 'double_dunk', 'surround', 'lost_luggage', 'hero', 'crazy_climber', 'krull', 'atlantis', 'keystone_kapers', 'bank_heist', 'skiing', 'phoenix', 'venture', 'ice_hockey', 'robotank', 'fishing_derby', 'demon_attack', 'elevator_action', 'bowling', 'zaxxon', 'gravitar', 'kung_fu_master', 'amidar', 'kangaroo', 'star_gunner', 'trondead', 'name_this_game', 'adventure', 'kaboom', 'beam_rider', 'king_kong', 'video_pinball', 'enduro', 'up_n_down', 'breakout', 'pong', 'centipede', 'tennis', 'koolaid', 'air_raid', 'defender', 'montezuma_revenge', 'riverraid', 'alien', 'chopper_command', 'qbert', 'donkey_kong']

In [7]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

print(f'Game Frame Dimensions: [{height},{width},{channels}]')
print(f'Number of Actions in Game: {actions}')
print(f'Details of available actions in Game: {env.unwrapped.get_action_meanings()}')

Game Frame Dimensions: [210,160,3]
Number of Actions in Game: 6
Details of available actions in Game: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SpaceInvaders-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which ret

In [8]:
episodes = 5

for episode in range(episodes):
    score = 0
    done = False
    state = env.reset()
    while not done:
        frame = env.render(mode='rgb_array')
        action_to_take = random.choice([0,1,2,3,4,5])
        new_state, reward, done, info = env.step(action_to_take)
        score+=reward
    print(f'Episode {episode} -> Score {score}')

env.reset()
env.close()

/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(done, 

Episode 0 -> Score 180.0
Episode 1 -> Score 140.0
Episode 2 -> Score 90.0
Episode 3 -> Score 110.0
Episode 4 -> Score 60.0


In [9]:
def QNet(height, width, channels, actions):
    init = RandomNormal(stddev=0.02)

    in_src_image = Input(shape=(height, width, channels))

    d = Conv2D(8, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_src_image)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(16, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(32, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Flatten()(d)
    d = Dense(512, activation='relu')(d)
    d = Dense(128, activation='relu')(d)

    out = Dense(actions, activation='linear')(d)

    model = Model(in_src_image, out)

    opt = Adam(learning_rate=0.0002)
    model.compile(loss='mse', optimizer=opt)

    return model

q_net = QNet(height, width, channels, actions)
q_net.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 210, 160, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 105, 80, 8)        392       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 105, 80, 8)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 53, 40, 16)        2064      
                                                                 
 batch_normalization (Batch  (None, 53, 40, 16)        64        
 Normalization)                                                  
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 53, 40, 16)        0         
                                                             

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


 conv2d_4 (Conv2D)           (None, 7, 5, 64)          65600     
                                                                 
 batch_normalization_3 (Bat  (None, 7, 5, 64)          256       
 chNormalization)                                                
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 5, 64)          0         
                                                                 
 flatten (Flatten)           (None, 2240)              0         
                                                                 
 dense (Dense)               (None, 512)               1147392   
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dense_2 (Dense)             (None, 6)                 774       
                                                                 
Total para

In [10]:
class DDQNAgent:
    def __init__(self, state_shape, action_size, model_fn, gamma=0.99, learning_rate=0.0001,
                 epsilon_start=1.0, epsilon_min=0.1, epsilon_decay=0.995, replay_buffer_size=2000, batch_size=64):
        self.state_shape = state_shape
        self.action_size = action_size
        self.gamma = gamma
        self.learning_rate = learning_rate

        self.epsilon = epsilon_start
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

        self.replay_buffer = deque(maxlen=replay_buffer_size)
        self.batch_size = batch_size

        self.model = model_fn(*state_shape, action_size)
        self.target_model = model_fn(*state_shape, action_size)
        self.update_target_model()

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(np.expand_dims(state, axis=0), verbose=0)
        return np.argmax(q_values[0])

    def store_experience(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))

    def train(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        minibatch = random.sample(self.replay_buffer, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*minibatch)

        states = np.array(states)
        next_states = np.array(next_states)

        q_next_online = self.model.predict(next_states, verbose=0)
        q_next_target = self.target_model.predict(next_states, verbose=0)

        q_targets = self.model.predict(states, verbose=0)
        for i in range(self.batch_size):
            if dones[i]:
                q_targets[i][actions[i]] = rewards[i]
            else:
                best_action = np.argmax(q_next_online[i])
                q_targets[i][actions[i]] = rewards[i] + self.gamma * q_next_target[i][best_action]

        self.model.fit(states, q_targets, epochs=1, verbose=0, batch_size=self.batch_size)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [11]:
state_shape = (height, width, channels)
agent = DDQNAgent(state_shape=state_shape, action_size=actions, model_fn=QNet)

episodes = 3
sync_target_model_freq = 10

In [12]:
for episode in range(1, episodes + 1):
    state = env.reset()
    score = 0
    done = False

    while not done:
        action = agent.act(state)

        next_state, reward, done, _ = env.step(action)
        score += reward

        agent.store_experience(state, action, reward, next_state, done)

        agent.train()

        state = next_state

    if episode % sync_target_model_freq == 0:
        agent.update_target_model()

    print(f"Episode {episode}/{episodes} | Score: {score} | Epsilon: {agent.epsilon:.4f}")

env.close()

Episode 1/3 | Score: 235.0 | Epsilon: 0.0997
Episode 2/3 | Score: 210.0 | Epsilon: 0.0997
Episode 3/3 | Score: 240.0 | Epsilon: 0.0997
